## Week 5 assignment

#### Task

In the [course material](https://github.com/kopuj/neuralNetworks/blob/master/Attention.ipynb), a simple binary classifier containing an attention layer has been built and trained. Using this model as a starting point, you should modify it by adding

- a residual connection (around the attention layer), and
- layer normalization (directly after the attention layer),

as shown in the image in section 11.4.3 (The Transformer encoder) in Chollet's "Deep Learning with Python", 2nd ed. Use the functional API to implement the residual connection, and the LayerNormalization layer provided by Keras. (You do not need to construct a complete Transformer encoder, the two elements mentioned above are enough.)

Also, train the model, and test it with the test set.


In [7]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import logging; logging.getLogger('tensorflow').setLevel(logging.ERROR)

#### Data Import and Prep

In [8]:
from keras.datasets import imdb
from keras.utils import pad_sequences

max_features = 10000
max_len = 250

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=max_len, padding="post")
x_test = pad_sequences(x_test, maxlen=max_len, padding="post")

#### Indexing and Sample of Data

In [9]:
word_to_index = imdb.get_word_index()
index_to_word = dict((value, key) for (key, value) in word_to_index.items())

example_review = " ".join(index_to_word.get(i - 3, "?") for i in x_train[0])
print(example_review)

? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

#### TokenPositionEmbedding layer

In [10]:
from keras.layers import Layer, Embedding
import tensorflow as tf


class TokenAndPositionEmbedding(Layer):
    def __init__(self, seq_len, vocab_size, emb_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=emb_dim)
        self.pos_emb = Embedding(input_dim=seq_len, output_dim=emb_dim)

    def call(self, x_input):
        seq_len = tf.shape(x_input)[-1]
        positions = tf.range(start=0, limit=seq_len, delta=1)
        positions = self.pos_emb(positions)
        x_input = self.token_emb(x_input)
        return x_input + positions

#### Final Model

In [11]:
from keras.layers import (
    MultiHeadAttention,
    LayerNormalization,
    Add,
    Input,
    Dense,
    GlobalAveragePooling1D,
    Dropout,
)
from keras.models import Model

embed_dim = 32

num_heads = 2  # number of attention heads
key_dim = embed_dim // num_heads  # query/key dimension for one head

inputs = Input(shape=(max_len,))
x = TokenAndPositionEmbedding(max_len, max_features, embed_dim)(inputs)

attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x, x)
residual = Add()([x, attention_output])
x = LayerNormalization()(residual)

ffn_output = Dense(embed_dim, activation="relu")(x)
x = Add()([x, ffn_output])
x = LayerNormalization()(x)

x = GlobalAveragePooling1D()(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

att_model = Model(inputs=inputs, outputs=outputs)
att_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
att_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 250)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ token_and_position… │ (None, 250, 32)   │    328,000 │ input_layer_1[0]… │
│ (TokenAndPositionE… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 250, 32)   │      4,224 │ token_and_positi… │
│ (MultiHeadAttentio… │                   │            │ token_and_positi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 250, 32)   │          0 │ token_and_positi… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 250, 32)   │         64 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 250, 32)   │      1,056 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 250, 32)   │          0 │ layer_normalizat… │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 250, 32)   │         64 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 32)        │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         33 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 333,441 (1.27 MB)

 Trainable params: 333,441 (1.27 MB)

 Non-trainable params: 0 (0.00 B)

#### Final Model Training

In [12]:
att_model.fit(x_train, y_train, epochs=10, batch_size=1024)


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 181ms/step - accuracy: 0.5480 - loss: 0.7539
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.7112 - loss: 0.6015
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.7750 - loss: 0.5467
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.8139 - loss: 0.4879
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.8511 - loss: 0.3921
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.8767 - loss: 0.3064
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.9104 - loss: 0.2363
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.9302 - loss: 0.1942
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.9374 - loss: 0.1753
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.9448 - loss: 0.1549


#### Final Model Evaluation

In [13]:
print(f'Test accuracy = {att_model.evaluate(x_test, y_test)[1]:.4f}')


782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8714 - loss: 0.3344
Test accuracy = 0.8685
